### GPU 설정

In [1]:
import tensorflow as tf 
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15524708427629997363
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2909221684
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4863385509044985658
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# LeNet

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, AveragePooling2D, Flatten, Dense, ZeroPadding2D

from sklearn.model_selection import train_test_split

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [5]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

In [6]:
LeNet = Sequential([InputLayer(input_shape = (28, 28, 1)),   # input layer
                    
                    ZeroPadding2D((2,2)),
                    
                    Conv2D(6, 5, activation='tanh'),
                    AveragePooling2D(strides=2),

                    Conv2D(16, 5, activation='tanh'),
                    AveragePooling2D(strides=2),

                    Conv2D(120, 5, activation='tanh'),
                    Flatten(),

                    Dense(84, activation='tanh'),
                    Dense(10, activation='softmax')                    
                    ])

In [7]:
LeNet.compile(optimizer='SGD',
              loss = 'sparse_categorical_crossentropy',
              metrics = 'accuracy')

In [8]:
LeNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPaddin  (None, 32, 32, 1)         0         
 g2D)                                                            
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (Average  (None, 14, 14, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Avera  (None, 5, 5, 16)          0         
 gePooling2D)                                                    
                                                        

In [10]:
LeNet.fit(x_train, y_train, epochs=3)

Epoch 1/3
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2921 - accuracy: 0.9144
Epoch 2/3
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2303 - accuracy: 0.9322
Epoch 3/3
1500/1500 [==============================] - 8s 5ms/step - loss: 0.1877 - accuracy: 0.9449


In [14]:
LeNet.evaluate(x_val, y_val)

375/375 [==============================] - 1s 3ms/step - loss: 0.1808 - accuracy: 0.9459


[0.1807662397623062, 0.9459166526794434]

# AlexNet

In [3]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

데이터 불러오기

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory = 'D:/dogs_vs_cats/dogs_vs_cats/train/',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (227, 227)
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    directory = 'D:/dogs_vs_cats/dogs_vs_cats/test/',
    labels = 'inferred',   # labels are generated from the directory structure
    label_mode = 'int',    # 0 = cats, 1 = dogs
    batch_size = 32,
    image_size = (227, 227)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


데이터 정규화

In [5]:
# Normalize
def process(image, label):
    image = tf.cast(image / 255., tf.float32)
    return image, label

train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

모델 선언

In [6]:
model = Sequential(name='alex_net')
model.add(Conv2D(filters=96, kernel_size=11,strides=4,activation='relu',input_shape=(227,227,3)))
model.add(MaxPool2D(pool_size=3,strides=2))

model.add(Conv2D(filters=256, kernel_size=5, activation='relu',padding="same"))
model.add(MaxPool2D(pool_size=3,strides=2))

model.add(Conv2D(filters=384, kernel_size=3,strides=1,activation='relu',padding="same"))
model.add(Conv2D(filters=384,kernel_size=3,strides=1,activation='relu',padding="same"))
model.add(Conv2D(filters=256,kernel_size=3,strides=1,activation='relu',padding="same"))
model.add(MaxPool2D(pool_size=3,strides=2))

model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5)) # dropout : 보통 Dense layer part에서 진행 
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [7]:
model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10


: 

In [3]:
import tensorflow.keras as keras

# VGG16

In [ ]:
import os
import os.path
from pathlib import Path
from keras.preprocessing.image import ImageDataGenerator

import pandas as pd

import tensorflow as tf

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory = 'D:/dogs_vs_cats/dogs_vs_cats/train/',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (224, 224)
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    directory = 'D:/dogs_vs_cats/dogs_vs_cats/test/',
    labels = 'inferred',   
    label_mode = 'int',    
    batch_size = 32,
    image_size = (224, 224)
)

In [ ]:
# train_path = Path("D:/Brain_Tumor_Classification(MRI)/Training")
# test_path = Path("D:/Brain_Tumor_Classification(MRI)/Testing")
train_path = Path("D:/dogs_vs_cats/dogs_vs_cats/train/")
test_path = Path("D:/dogs_vs_cats/dogs_vs_cats/test/")

train_jpg_path = list(train_path.glob(r"*/*.jpg"))
test_jpg_path = list(test_path.glob(r"*/*.jpg"))

train_jpg_labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],train_jpg_path))
test_jpg_labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],test_jpg_path))

train_jpg_path_series = pd.Series(train_jpg_path,name="JPG").astype(str)
train_jpg_labels_series = pd.Series(train_jpg_labels,name="CATEGORY").astype(str)

test_jpg_path_series = pd.Series(test_jpg_path,name="JPG").astype(str)
test_jpg_labels_series = pd.Series(test_jpg_labels,name="CATEGORY").astype(str)

main_train_data = pd.concat([train_jpg_path_series, train_jpg_labels_series],axis=1)
main_test_data = pd.concat([test_jpg_path_series, test_jpg_labels_series],axis=1)
main_train_data = main_train_data.sample(frac=1).reset_index(drop=True)
main_test_data = main_test_data.sample(frac=1).reset_index(drop=True)

train_img_generator = ImageDataGenerator(rescale=1./255,
                                         rotation_range=25,
                                         brightness_range=[0.3,0.7],
                                         width_shift_range=0,
                                         height_shift_range=0,
                                         zoom_range=0)

test_img_generator = ImageDataGenerator(rescale=1./255,
                                        rotation_range=25,
                                        brightness_range=[0.3,0.7],
                                        width_shift_range=0,
                                        height_shift_range=0,
                                        zoom_range=0)

train_ds = train_img_generator.flow_from_dataframe(dataframe=main_train_data,
                                                   x_col="JPG",
                                                   y_col="CATEGORY",
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   batch_size=10,
                                                   subset="training",
                                                   target_size=(227,227))

test_ds = test_img_generator.flow_from_dataframe(dataframe=main_test_data,
                                                 x_col="JPG",
                                                 y_col="CATEGORY",
                                                 color_mode='rgb',
                                                 class_mode="categorical",
                                                 batch_size=10,
                                                 subset="training",
                                                 target_size=(227,227))

train_valid =train_img_generator.flow_from_dataframe(dataframe=main_train_data,
                                                     x_col="JPG",
                                                     y_col="CATEGORY",
                                                     color_mode="rgb",
                                                     class_mode="categorical",
                                                     batch_size=10,
                                                     subset="training",
                                                     target_size=(227,227))

test_valid =test_img_generator.flow_from_dataframe(dataframe=main_test_data,
                                                   x_col="JPG",
                                                   y_col="CATEGORY",
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   batch_size=10,
                                                   subset="training",
                                                   target_size=(227,227))

In [ ]:
vgg = Sequential(name="vgg16")

# input
vgg.add(Conv2D(64,3, strides=1, activation="relu", padding="same", input_shape=(224, 224, 3)))
vgg.add(Conv2D(64,3, strides=1, activation="relu", padding="same"))

vgg.add(MaxPool2D(2,2))

vgg.add(Conv2D(128,3, strides=1, activation="relu", padding="same"))
vgg.add(Conv2D(128,3, strides=1, activation="relu", padding="same"))

vgg.add(MaxPool2D(2,2))

vgg.add(Conv2D(256,3, strides=1, activation="relu", padding="same"))
vgg.add(Conv2D(256,3, strides=1, activation='relu', padding="same"))
vgg.add(Conv2D(256,1, strides=1, activation='relu', padding="same"))

vgg.add(MaxPool2D(2,2))

vgg.add(Conv2D(512,3, strides=1, activation="relu", padding="same"))
vgg.add(Conv2D(512,3, strides=1, activation='relu', padding="same"))
vgg.add(Conv2D(512,1, strides=1, activation='relu', padding="same"))

vgg.add(MaxPool2D(2,2))

vgg.add(Conv2D(512,3, strides=1, activation="relu", padding="same"))
vgg.add(Conv2D(512,3, strides=1, activation='relu', padding="same"))
vgg.add(Conv2D(512,1, strides=1, activation='relu', padding="same"))

vgg.add(MaxPool2D(2,2))

# Flatten Layer
vgg.add(Flatten())
# FC Layer
vgg.add(Dense(4096,activation='relu'))
vgg.add(Dense(4096,activation='relu'))
## vgg.add(Dense(1000,activation='relu')) # 몇 개로 분류할 건지 
# vgg.add(Dense(4, activation="softmax")) # 3개 이상 softmax
vgg.add(Dense(1, activation="sigmoid")) 

In [ ]:
vgg.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]) # loss="sparse_categorical_crossentropy",
vgg.summary()

In [ ]:
vgg.fit(train_ds, train_valid, epochs=10)